In [ ]:
'''
fasttext로 tag데이터 학습
1.tag데이터 
2.오탈자체크 교정
3.tag->가장 가까운 post 테스트
'''

### import

In [40]:
import gensim
print(gensim.__version__)
from soynlp.hangle import decompose
from soynlp.hangle import compose
import pandas as pd
import numpy as np

3.4.0


### util

In [6]:
# 한글 자모음 분리
def preprocessing(s):
    s_ = ' '
    for c in s:
        if c == ' ' and s_[-1] != ' ':
            s_ += ' '
            continue
        jamo = decompose(c)
        if (not jamo):
            if s_[-1] != ' ':
                s_ += ' '
            continue
        if (len(jamo) != 3) or (jamo[0] == ' ' or jamo[1] == ' '):
            s_ += ' '
            continue        
        s_ += ''.join([jamo[0], jamo[1], '-' if jamo[2] == ' ' else jamo[2]])
    return s_.strip()

# 한글 자모음 조합
def recover(s, debug=False):

    assert len(s) % 3 == 0

    s_ = s.replace('-', ' ')
    chars = [tuple(s_[3*i:3*(i+1)]) for i in range(len(s_)//3)]

    if debug:
        print(chars)

    recovered = [compose(*char) for char in chars]
    recovered = ''.join(recovered)
    return recovered

In [22]:
preprocessing("우하하핳켘켘아이고오오고")

'ㅇㅜ-ㅎㅏ-ㅎㅏ-ㅎㅏㅎㅋㅔㅋㅋㅔㅋㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-ㅇㅗ-ㄱㅗ-'

In [23]:
recover("ㅇㅜ-ㅎㅏ-ㅎㅏ-ㅎㅏㅎㅋㅔㅋㅋㅔㅋㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-ㅇㅗ-ㄱㅗ-")

'우하하핳켘켘아이고오오고'

### data load

In [26]:
data1 = pd.read_csv("../data/ta_hashtag/doc2vec_ta_hashtag_0-10M.csv", header=None, encoding='CP949')
data2 = pd.read_csv("../data/ta_hashtag/doc2vec_ta_hashtag_10-20M.csv", header=None, encoding='CP949')
data3 = pd.read_csv("../data/ta_hashtag/doc2vec_ta_hashtag_20-30M.csv", header=None, encoding='CP949')
data4 = pd.read_csv("../data/ta_hashtag/doc2vec_ta_hashtag_30-40M.csv", header=None, encoding='CP949')
data5 = pd.read_csv("../data/ta_hashtag/doc2vec_ta_hashtag_40-47M.csv", header=None, encoding='CP949')

In [27]:
# merge
data = pd.concat([data1,data2,data3,data4,data5])

In [28]:
data.shape

(47731282, 2)

### preprocessing

In [30]:
data[1][:4]

0                                                   패션
1                                                   패션
2    패션,데일리룩,셀스타그램,거울샷,셀카,셀피,아이폰,오오티디,니트,터틀넥니트,like...
3                                   패션,commedesgarcons
Name: 1, dtype: object

In [43]:
%%time
tag_devided = data[1].map(lambda x: preprocessing(x.replace(',',' ')))

Wall time: 50min 58s


In [45]:
tag_devided[:5]

0                                               ㅍㅐ-ㅅㅕㄴ
1                                               ㅍㅐ-ㅅㅕㄴ
2    ㅍㅐ-ㅅㅕㄴ ㄷㅔ-ㅇㅣㄹㄹㅣ-ㄹㅜㄱ ㅅㅔㄹㅅㅡ-ㅌㅏ-ㄱㅡ-ㄹㅐㅁ ㄱㅓ-ㅇㅜㄹㅅㅑㅅ ...
3                                               ㅍㅐ-ㅅㅕㄴ
4    ㅍㅐ-ㅅㅕㄴ ㅅㅔㄹㅋㅏ- ㅅㅔㄹㅍㅣ- ㅅㅔㄹㅅㅡ-ㅌㅏ-ㄱㅡ-ㄹㅐㅁ ㅁㅗ-ㄷㅔㄹ ㅍㅣ...
Name: 1, dtype: object

In [44]:
savePath = '../data/ta_hashtag/fasttext_tag_devided.txt'
np.savetxt(savePath, tag_devided.values, delimiter=' ', fmt='%s')

### fasttext

In [50]:
from gensim.models import Sent2Vec

ImportError: cannot import name 'Sent2Vec'